# Just-in-Time компиляция

«Как раз вовремя» — компиляция, при которой машинный код геенрируется по мере исполнения программы для её критических с точки зрения производительности участков — обычно функций, реже — циклов.

Компиляция происходит при первом вызове, поэтому для того, чтобы измерить производительность JIT, следует сперва вызвать функцию «вхолостую», можно на данных небольшого объёма.

Некоторые реализации Python ([PyPy](https://www.pypy.org/)) содержат JIT-компилятор в дистрибутиве. Для CPython можно воспользоваться пакетом [Numba](https://numba.pydata.org/): `pip install numba`.

In [7]:
from __future__ import annotations
from numba import jit, prange
import numpy
import numpy.random

random_m = numpy.matrix(numpy.random.rand(300, 30000))
heat_m = numpy.zeros((1, 1))

def sum_matrix_nojit(m: numpy.matrix)-> numpy.float:
    s = 0.0
    d0, d1 = m.shape
    for r in range(d0):
        for c in range(d1):
            s += m[r, c]
    return s


@jit(nopython=True, fastmath=True)
def sum_matrix_jit(m: numpy.matrix)-> numpy.float:
    s = 0.0
    d0, d1 = m.shape
    for r in range(d0):
        for c in range(d1):
            s += m[r, c]
    return s


@jit(nopython=True, parallel=True, fastmath=True)
def sum_matrix_jit_par(m: numpy.matrix)-> numpy.float:
    s = 0.0
    d0, d1 = m.shape
    for r in prange(d0):
        sc = 0.0
        for c in range(d1):
            sc += m[r, c]
        s += sc
    return s

In [8]:
%timeit sum_matrix_nojit(random_m)

sum_matrix_jit(heat_m)  # give it a heat
%timeit sum_matrix_jit(random_m)

sum_matrix_jit_par(heat_m)  # give it a heat
%timeit sum_matrix_jit_par(random_m)

9.38 s ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.98 ms ± 94.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.83 ms ± 422 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
